In [2]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [3]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph

from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver() # 그래프의 기억력이 유지하도록 한다. postgresMemorySaver 는 글로벌!

class State(TypedDict):
    messages: Annotated[list, add_messages]

tool = TavilySearchResults(max_results=2)
tools = [tool]
tool_node = ToolNode(tools)

llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    result = llm_with_tools.invoke(state["messages"])
    return {"messages": [result]}

graph_builder = StateGraph(State)

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)

graph_builder.add_edge("tools", "chatbot")
graph_builder.add_conditional_edges("chatbot", tools_condition)

graph_builder.set_entry_point("chatbot")
graph = graph_builder.compile(checkpointer=memory) # 특정 스레드에 한해서 대화의 기억이 유지됨

In [4]:
# 대화의 스레드를 id 로 지정 id ㄱㅏ 유지되는 한 기억을 지속할 수 있음
config = {"configurable": {"thread_id": "1"}}
while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")
        break
    for event in graph.stream({"messages": ("user", user_input)}, config):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)

Assistant: 
Assistant: [{"url": "https://namu.wiki/w/%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD%20%EB%8C%80%ED%86%B5%EB%A0%B9", "content": "대한민국의 현직 대통령은 제20대 윤석열이며, 임기는 2022년 5월 10일부터 2027년 5월 9일까지이다."}, {"url": "https://ko.wikipedia.org/wiki/%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD_%EB%8C%80%ED%86%B5%EB%A0%B9", "content": "편집 대통령은 헌법과 법률이 정하는 바에 의하여 국군을 통수한다(헌법 제74조).[9] 다만 군사에 관한 중요사항의 경우 국가안전보장회의의 자문과 국무회의의 심의를 거쳐 국무총리와 관계 국무위원이 부서한 문서로 하여야 하며(헌법 제91조, 제89조 제6호, 제82조), 선전포고나 국군의 외국 파견 등에는 국회의 사전 동의를 필요로 한다(헌법 제60조 제2항). 대통령은 국무회의의 심의를 거쳐(헌법 제89조 제3호) 헌법개정안을 발안할 수 있다(헌법 제128조 제1항). 헌법 제79조 제1항은 “대통령은 법률이 정하는 바에 의하여 사면·감형 또는 복권을 명할 수 있다”, \"일반 사면은 국회의 동의를 거쳐야 한다\"고 하여 대통령의 사면권을 규정하고 있으며, 이에 따라 사면법이 정해져 있다.[12] 대한민국의 대통령 권한대행[편집] 이 직위는 대한민국 헌법 제71조 \"대통령이 궐위되거나 사고로 인하여 직무를 수행할 수 없을 때에는 국무총리, 법률이 정한 국무위원의 순서로 그 권한을 대행한다.\"에 규정되어 있다."}]
Assistant: 현재 대한민국의 대통령은 제20대 윤석열이며, 그의 임기는 2022년 5월 10일부터 2027년 5월 9일까지입니다. 더 자세한 정보는 [여기](https://namu.wiki/w/%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD%20%EB%8C%80%E

KeyboardInterrupt: Interrupted by user

In [5]:
snapshot = graph.get_state(config)
snapshot

StateSnapshot(values={'messages': [HumanMessage(content='대한민국 대통령은 누구야?', additional_kwargs={}, response_metadata={}, id='de6a069c-1e85-4f94-9ab7-70812409d71d'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_jncqfyjC2abSXBRFnG3cIcUa', 'function': {'arguments': '{"query":"대한민국 대통령"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 86, 'total_tokens': 107, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-78dc8eac-7075-42ea-b998-552dbec5cb1a-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '대한민국 대통령'}, 'id': 'call_jncqfyjC2abSXBRFnG3cIcUa', 'type'

In [6]:
config2 = {"configurable": {"thread_id": "2"}}
graph.invoke({"messages": [{"role": "user", "content": "내가 한 첫 질문이 뭐였어?"}]},
             config2)

{'messages': [HumanMessage(content='내가 한 첫 질문이 뭐였어?', additional_kwargs={}, response_metadata={}, id='29f2ee14-d86d-4f3c-b011-f24f80dc4ece'),
  AIMessage(content='죄송하지만, 이전 대화 내용을 확인할 수 없기 때문에 당신이 한 첫 질문을 알 수 없습니다. 질문 내용을 다시 말씀해 주시면 도움이 될 수 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 89, 'total_tokens': 127, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-d22baad7-8be3-41f8-a3c6-c2237282e125-0', usage_metadata={'input_tokens': 89, 'output_tokens': 38, 'total_tokens': 127, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}

In [7]:
# 기억할 메세지 개수 제한하기

# 가장 마지막 함수 2개만 기억
def filter_messages(messages: list):
    # This is very simple helper function which only ever uses the last message
    return messages[-2:]

def chatbot(state: State):
    messages = filter_messages(state["messages"])
    result = llm_with_tools.invoke(messages)
    return {"messages": [result]}

graph_builder = StateGraph(State)

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)

graph_builder.add_edge("tools", "chatbot")
graph_builder.add_conditional_edges("chatbot", tools_condition)

graph_builder.set_entry_point("chatbot")
graph = graph_builder.compile(checkpointer=memory)

In [8]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "20"}}
input_message = HumanMessage(content="hi! I'm bob and I like soccer")
for event in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

# This will now not remember the previous messages
# (because we set `messages[-1:]` in the filter messages argument)
input_message = HumanMessage(content="what's my name?")
for event in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

input_message = HumanMessage(content="what's my name?")
for event in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

input_message = HumanMessage(content="what's my favorite?")
for event in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

hi! I'm bob and I like soccer
================================== Ai Message ==================================

Hi Bob! That's great to hear! Soccer is a fantastic sport with a lot of excitement. Do you have a favorite team or player?
================================ Human Message =================================

what's my name?
================================== Ai Message ==================================

Your name is Bob. How can I assist you today?
================================ Human Message =================================

what's my name?
================================== Ai Message ==================================

Your name is Bob. How can I assist you today?
================================ Human Message =================================

what's my favorite?
================================== Ai Message ==================================

I don't have any information on your personal p